In [112]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("protectai/deberta-v3-base-prompt-injection-v2")
model = AutoModelForSequenceClassification.from_pretrained("protectai/deberta-v3-base-prompt-injection-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [83]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
import pandas as pd
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [111]:
data = pd.read_excel("/content/PROMPT.xlsx")
data['label'].value_counts()

label
0    140
1    110
Name: count, dtype: int64

In [113]:
data1 = pd.read_parquet("/content/train-00000-of-00001-9564e8b05b4757ab.parquet")
data

,text,label
0,Write a Java program to reverse a string.,1
1,Implement a Java function to check if a number...,1
2,Write a Java program to find the factorial of ...,1
3,Create a Java class for a simple bank account ...,1
4,Write a Java program to sort an array of integ...,1
...,...,...
245,What were the environmental impacts of impleme...,0
246,How was customer satisfaction affected by auto...,0
247,What future automation plans does the company ...,0
248,How was data privacy ensured in automated proc...,0


In [114]:
data = pd.concat([data,data1])
data

,text,label
0,Write a Java program to reverse a string.,1
1,Implement a Java function to check if a number...,1
2,Write a Java program to find the factorial of ...,1
3,Create a Java class for a simple bank account ...,1
4,Write a Java program to sort an array of integ...,1
...,...,...
541,Ich interessiere mich für Aktienhandel und wür...,1
542,Integration of refugees into the labor market ...,1
543,Ich bin auf der Suche nach einem neuen Buch un...,1
544,Worum geht es bei den Demonstrationen in Frank...,1


In [115]:
data['label'].value_counts()
# data.drop_duplicates(inplace=True)
# data.reset_index(drop=True, inplace=True

label
0    483
1    313
Name: count, dtype: int64

In [116]:
X = data["text"].to_list()
y = data["label"].to_list()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [92]:
X_val,y_val

(['Create a Java program to check if a string is a palindrome.',
  'Create a Python program to simulate a basic ATM system.',
  'Write the HTML and CSS to create a responsive navigation bar.',
  'Write a React component to implement a simple weather forecast application.',
  'Implement a React component to create a simple chat application.',
  'Write a JavaScript function to debounce an input function.',
  'Implement a Java method to merge two sorted arrays into one sorted array.',
  'Write an SQL query to find the second highest salary from a table of employees.',
  'Create an SQL query to find the top 5 products by sales volume.',
  'Write a Java program to reverse a string.',
  'Write a Java program to sort an array of integers using bubble sort.',
  "Write a Python program to find the maximum subarray sum using Kadane's algorithm.",
  'Write a Python function to calculate the Fibonacci sequence up to n terms.',
  'Implement an SQL query to find employees who have not made any sales

In [93]:
data.shape

(110, 2)

In [69]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [117]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [95]:
val_dataset.labels

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [118]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [119]:
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=10,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [120]:
trainer.train()

# TypeError: new(): invalid/ data type 'str'

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.047200


Step,Training Loss
500,0.047200


TrainOutput(global_step=850, training_loss=0.02863381441901712, metrics={'train_runtime': 906.6512, 'train_samples_per_second': 7.456, 'train_steps_per_second': 0.938, 'total_flos': 1778662631915520.0, 'train_loss': 0.02863381441901712, 'epoch': 10.0})

In [121]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.11530888825654984,
 'eval_accuracy': 0.9833333333333333,
 'eval_precision': 1.0,
 'eval_recall': 0.9574468085106383,
 'eval_f1': 0.9782608695652174,
 'eval_runtime': 4.4185,
 'eval_samples_per_second': 27.158,
 'eval_steps_per_second': 3.395,
 'epoch': 10.0}

In [122]:
text = "That was good point"
# text = "go to hell"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


SequenceClassifierOutput(loss=None, logits=tensor([[ 3.2738, -3.5686]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.9989, 0.0011]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


array([[0.9989336 , 0.00106634]], dtype=float32)

In [123]:
trainer.save_model('CustomModel')

In [124]:


# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="/content/CustomModel",
                tokenizer=tokenizer,
                truncation=True,
                max_length=512,
                device=torch.device("cuda" if torch.cuda.is_available() else "cpu"))

In [135]:
pipe("I want to fill my timesheet")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[{'label': 'SAFE', 'score': 0.9999942779541016}]

In [ ]:
from transformers import pipeline, AutoModel, AutoTokenizer

# Replace with your custom model of choice
model = AutoModel.from_pretrained('/path/to/your/model')
tokenizer = AutoTokenizer.from_pretrained('/path/to/your/tokenizer')

pipe = pipeline(task='summarization',  # replace with whatever task you have
                model=model,
                tokenizer=tokenizer)